# Numerical Methods for Option Pricing

I wanted to explore the three following approaches to pricing a vanilla European Option:

1. Binomial Trees
2. Numerically solving the PDE using finite differences
3. Simulating paths using Monte Carlo
    
I will then compare these to the analytical solution.

## Analytical Solution

The Black-Scholes-Merton PDE models the price of a European option, $V(S;t)$, as a function of a non-dividend paying risky underlying asset, $S$, and time $t$.

$ \frac{\partial V}{\partial t} + \frac{1}{2}\sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} + rS\frac{\partial V}{\partial S}- rV =0 $

The price of a call option can be obtained using the boundary condition: $V(S_T;T) = \max[S_T-K, 0]$, where $T$ is the time to expiry and $K$ is the strike price.

This gives the solution:

$V(S_0; 0) = S_0\Phi(d_1)-Ke^{-rT}\Phi(d_2)$, where

$d_1 = \frac{\ln(S_0/K)+(r+\sigma^2/2)T}{\sigma\sqrt{T}}$, $d_2=d_1-\sigma\sqrt{T}$, $S_0$ is the initial price of the asset and $\Phi(x)$ is the cumulative Normal distribution function. 

The model has the following assumptions:

* **The asset price follows Geometric Brownian Motion (GBM) whose percentage returns have a constant drift $\mu$ and volatility $\sigma$.** Apparently it was [Mandelbrot](https://is.muni.cz/el/1456/podzim2009/MPM_AEMM/um/Mandelbrot_-_Forecasts_Of_Future_Price_Unbiased_Markets_And_Martingale_Model.pdf) who first suggested that asset prices be modelled as martingales. It is then perhaps no surprise that his student, Eugene Fama, is seen as the father of the Efficient Market Hypothesis (EMH). The weak form of EMH is essentially the economic analogue of Mandelbrot's mathematical insight. That is, historical prices are useless at predicting the future and instead the current asset price reflects all available information. If we assume that prices are martingales then Levy’s Characterization Theorem suggests that we should expect them to follow some time-changed Brownian Motion, underlining the ubiquity of this model. Of course, standard Brownian Motion (with no time change) is not a suitable model as it could predict negative asset prices. In addition, the expected percentage return on an investment would not be independent of the underlying asset price - as one experiences in real markets. For example, we observe no relationship between annual percentage returns on real assets and their absolute asset price. GBM addresses both of these problems: it implies that the percentage returns follow a Normal Distribution whilst the asset price itself has a Log-Normal Distribution. This model obviously has its limitations - see Extensions at the end. 


* **Percentage returns are independent.** This is really just another assumption underlying the GBM model of asset prices but it is often overlooked. Another way to state this is that GBM is Markovian or *memoryless*. That is, the probability distribution of future prices is dependent only on the current price and not on any historical prices. Note that this is subtly different to being a martingale, which states that the expectation of future prices is dependent only on the current price (and is infact equal to it) and does not depend on any historical prices. In general neither property implies the other. The weak form of EMH is essentially a combination of the Markovian and martingale assumptions. The generally accepted phenomenon of *momentum* (the tendency for rising assets to continue rising and falling assets to continue falling) contradicts both of these assumptions. Such anomalies are best explained by the cognitive theorems of Behavioural Economics. Whilst a Markov process must be completely memoryless, a martingale need only have a memoryless expectation. For this reason, in the context of financial markets, the Markovian assumption seems more difficult to justify. The phenomenon of volatility clustering (the tendency for large changes to follow large changes and small changes to follow small changes) contradicts solely the Markovian assumption. Note that this was also first observed by [Mandelbrot](https://www.unc.edu/~fbaum/teaching/articles/MandelbrotCottonPrices1967.pdf).


* **Percentage returns are stationary.** This is also just another assumption underlying the GBM model of asset prices. A single definition of stationarity has never been quite decided upon by the mathematical community but in general a process is stationary if its statistical properties are invariant of time. In the context of option pricing this usually refers to the volatility of the asset. One can readily show that this is not constant in reality and many models have been proposed to confront this.


* **The market is efficient and so no arbitrage opportunities exist.** The PDE is derived assuming this and can then be solved using a suitable transformation. A much easier approach is to employ The Fundamental Theorems of Asset Pricing which imply that a unique *risk-neutral* probability measure exists if a market is complete (negligible transaction costs and a price for every asset) and arbitrage-free. One can then reach the solution of the PDE immediately by using this measure to calculate the discounted expectation of the option's payout. See the Monte Carlo section for further discussion on this.


* **There exists a riskless asset with a constant rate of return $r$.** This is typically taken as LIBOR.


* **$S$ is a continuous variable.** That is, asset prices are continuously divisible.


* **The asset is very liquid.** It is possible to buy or short any amount of the asset without impacting the market.


* **No transaction costs.**

The following function uses this analytical solution to price a European call option. For example, an option on an asset with initial price \$200, volatility 14% per annum ($\sigma = 0.14$), strike price \$205, time to maturity of six months ($T=0.5$) and a risk-free interest rate of 2% ($r=0.02$), should be priced at $6.56.

In [1]:
import numpy as np
import scipy.stats as ss

def d1(S0, K, r, sigma, T):
    return ( np.log(S0/K) + (r + sigma**2/2)*T ) / (sigma * np.sqrt(T))

def d2(S0, K, r, sigma, T):
    return d1(S0, K, r, sigma, T) - sigma*np.sqrt(T)

def BlackScholesCall(S0, K, r, sigma, T):
    return S0*ss.norm.cdf(d1(S0, K, r, sigma, T)) - K*np.exp(-r*T)*ss.norm.cdf(d2(S0, K, r, sigma, T))

In [2]:
BlackScholesCall(200, 205, 0.02, 0.14, 0.5)

6.5611219446153655

## Binomial Trees

#### Cox-Ross-Rubinstein model

This valuation divides the life of the option up into discrete time intervals of length $\Delta t$. The asset price $S$ is assumed to either move up to $Su$ (with probability $p$) or down to $Sd$ (with probability $1-p$) in any one step. The fractional move up is assumed the same as the fractional move down, that is $u=\frac{1}{d}$.

The three constants $p$, $u$ and $d$ can be found using a risk-neutral valuation. That is, set the expectation of the model after one time step equal to the asset price appreciated at the risk-free rate [ $S\exp(r\Delta t)$ ]. Finally set the variance of the model after one time step equal to $\sigma^2\Delta t$. By ignoring terms $\mathcal{O}(\Delta t^2)$ and higher one obtains the Cox-Ross-Rubinstein (CRR) pricing model:

$p=\frac{e^{r\Delta t} - d}{u-d}$, $u = e^{\sigma\sqrt{\Delta t}}$, $d = e^{-\sigma\sqrt{\Delta t}}$.

The price of the option can then be calculated by working backwards through the tree (see diagram below) and making many risk-neutral valuations. The value of the option at maturity is known, for example it is $\max[S_T-K, 0]$ for a call. Its value at $T-\Delta t$ is then the expectation of the model at $T$ discounted at the risk-free rate for a time $\Delta t$.

#### Implementation

Suppose we divide the life of the option up into $N$ subintervals of length $\Delta t$. The asset price at time $i\Delta t$ ($0\leq i\leq N$) in the $j$th node ($0\leq j \leq i$) is $S_0 u^j d^{i-j}$. Let $c(i,j)$ be the price of a call option at node $(i,j)$.

The price of the option for each possible asset price is known at expiration, giving:

$c(N,j) = \max(S_0 u^j d^{N-j} - K, 0)$ for $0\leq j \leq N$.

A recursive formula for the option price, using a risk-neutral valuation, at all previous times and asset prices is then:

$c(i,j) = e^{-r\Delta t}[pc(i+1,j+1) + (1-p)c(i+1, j)]$ where $0\leq i\leq N-1$ and $0\leq j \leq i$.

The following function performs this recursion, using an $N\times N$ triangular matrix to cache previous values. Pricing the same call option as before with $N=500$ results in a price of $6.56 - the same as the analytical solution to two decimal places.

#### Assumptions

One can show that in the limit as $N \rightarrow \infty$ the CRR price tends to that predicted by Black Scholes. The CRR model makes the same assumptions as Black-Scholes. A risk-neutral measure is presumed, which is equivalent to an arbitrage-free market, whilst the Binomial model assumed for percentage returns ($u$ and $d$) tends to a Normal distribution in the limit.

Note that there is no analytical solution for an American Option, but this can be handled easily in the CRR model. At each node you simply compare the predicted price of the option, if not exercised early, with the option's intrinsic value at that time if it were exercised early: selecting the maximum of these.

![alt text](CRR Tree.jpg)

In [4]:
import numpy as np
import math

def CRRCall(S0, K, r, sigma, T, N):
    
    delta_t = T/N;
        
    u = np.exp(sigma * np.sqrt(delta_t))
    d = np.exp(-sigma * np.sqrt(delta_t))
    p = (np.exp(r*delta_t) - d)/(u - d)
    
    c = [[0 for i in range(N+1)] for j in range(N+1)]
    
    for j in range(N+1):
        c[N][j] = max( S0*math.pow(u,j)*math.pow(d,N-j) - K, 0)
        
    for i in range(N-1, -1, -1):
        for j in range(i+1):
            c[i][j] = np.exp(-r * delta_t)*( p*c[i+1][j+1] + (1-p)*c[i+1][j] )
        
    return c[0][0]

In [5]:
CRRCall(200, 205, 0.02, 0.14, 0.5, 500)

6.5622533111014398

## Finite Differences

We will first discretise the continuous variables, assuming $N$ temporal intervals of length $\Delta t = \frac{T}{N}$ and $M$ price intervals of length $\Delta S = \frac{S_{\text{max}}}{M}$. Time is a bounded variable whereas the asset price could theoretically be infinite. For practical purposes I have had to introduce a maximum price, $S_{\text{max}}$, whose value will be discussed later. Let $c(i,j)$ be the price of a call option at time $i\Delta t$ ($0\leq i\leq N$) and underlying price $j\Delta S$ ($0\leq j\leq M$).

The price of the option for each possible asset price is known at expiration, giving:

$c(N,j) = \max(j\Delta S - K, 0)$ for $0\leq j \leq M$.

At any given moment in time, the option is worthless if the underlying is worthless:

$c(i,0) = 0$ for $0\leq i \leq N$.

In infinite space $c \rightarrow S$ as $S \rightarrow \infty$, thus in our finite discretisation:

$c(i,M) = S_{\text{max}}$ for $0\leq i \leq N$.

We will use a forward difference for the time derivative, a central difference for the first asset derivative and a mixture of forward and backward differences for the second asset derivative, giving:

$ [\frac{\partial c}{\partial t}]_{i,j} \approx \frac{c(i+1,j)-c(i,j)}{\Delta t} + \mathcal{O}(\Delta t)$,

$ [\frac{\partial c}{\partial S}]_{i,j} \approx \frac{c(i,j+1)-c(i,j-1)}{2\Delta S} + \mathcal{O}(\Delta S^2)$,

$ [\frac{\partial^2 c}{\partial S^2}]_{i,j} \approx \big([\frac{\partial c}{\partial S}]_{i,j+1} - [\frac{\partial c}{\partial S}]_{i,j}\big)/\Delta S + \mathcal{O}(\Delta S) = \big([\frac{c(i,j+1) - c(i,j)}{\Delta S}] - [\frac{c(i,j) - c(i,j-1)}{\Delta S}]\big)/\Delta S + \mathcal{O}(\Delta S^2) = \frac{c(i,j+1) - 2c(i,j) + c(i,j-1)}{\Delta S^2}+ \mathcal{O}(\Delta S^2)$

#### Implementation

The difficulty is that we need to work backwards and derive $c$ at $t=0$ from the boundary condition at $t=T$, but time flows forwards in Black-Scholes. Thus, we need to change variables and use the reverse time $t' = T - t$, giving the backwards PDE with the initial condition:

$ \frac{\partial V}{\partial t'} - \frac{1}{2}\sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} - rS\frac{\partial V}{\partial S}+ rV =0 $   where   $c(0,j) = \max(j\Delta S - K, 0)$ for $0\leq j \leq M$.

Substituting our finite differences into this gives:

$c(i+1,j) = \frac{1}{2}c(i,j-1)j\Delta t(\sigma^2j-r) + c(i,j)(1-r\Delta t -\sigma^2j^2\Delta t) + \frac{1}{2}c(i,j+1)j\Delta t(\sigma^2j+r) + \mathcal{O}(\Delta t + \Delta S^2)$

This is an explicit scheme in that the price of the option at $(i+1)\Delta t$ can be found explcitly in terms of its price at $i\Delta t$. This makes its implementation particularly simple, as compared to an implicit scheme or Crank-Nicolson. However, the explicit scheme is only stable (errors decay during iteration) if $\Delta t < (\sigma^2(M-1)+\frac{1}{2}r)^{-1}$, whereas the implicit schemes are unconditionally stable. There is no such thing as a free lunch.

In reverse time the price of the option is then found near $c(N,j_0)$ where $j_0 = \left \lfloor\frac{S_0}{\Delta S}\right \rfloor$. One could interpolate between $c(N,j_0)$ and $c(N,j_0+1)$ to obtain a better estimate.

Some experimentation is now required to find suitable values for $S_{\text{max}}$, $M$ and $N$. Setting $S_{\text{max}}$ to be five times $S_0$ seems to be adequate. Any price excursions beyond this are sufficiently unlikely that we can ignore their influence. The stability condition above that links $M$ and $N$ can make selecting their values a bit tricky, but setting $N$ to be an order of magnitude greater than $M$ seems to work well. Using a more robust unconditionally stable scheme would eliminate this interdependence.

Pricing the same call option as before with $N=5000$ and $M=500$ results in a price of $6.56 - the same as the analytical solution to two decimal places.

#### Assumptions

In numerically solving the Black-Scholes equation, finite difference methods are obviously subject to the same assumptions used to derive it. One can show that the explicit scheme above is actually equivalent to a Trinomial Tree approach.

Just as in tree based approaches, American Options can also easily be handled.

In [7]:
import math

def FinDiffCall(S0, K, r, sigma, T, N, M):
    
    delta_t = T/N;
    S_max = 5*S0;
    delta_S = S_max/M;
    
    if( delta_t > 1/( math.pow(sigma,2)*(M-1) + 0.5*r ) ):
        return "time resolution is not great enough"
        
    c = [[0 for j in range(M+1)] for i in range(N+1)]
    
    for j in range(M+1):
        c[0][j] = max( j*delta_S - K, 0)
    
    for i in range(N+1):
        c[i][0] = 0
        c[i][M] = S_max
        
    for i in range(1, N+1):
        for j in range(1, M):
            c[i][j] = 0.5*c[i-1][j-1]*j*delta_t*(math.pow(sigma,2)*j - r) + \
                        c[i-1][j] * (1 - r*delta_t - math.pow(sigma,2)*math.pow(j,2)*delta_t) + \
                            0.5*c[i-1][j+1]*j*delta_t*(math.pow(sigma,2)*j + r)
    
    return c[N][math.floor(S0/delta_S)]
        

In [8]:
FinDiffCall(200, 205, 0.02, 0.14, 0.5, 5000, 500)

6.561349162493725

## Monte Carlo

Monte Carlo methods are an explicit application of the Fundamental Theorems of Asset Pricing. If the market is complete and arbitrage-free then there is a unique measure under which the discounted asset price is a martingale: $\text{E}[e^{-rt}S(t)] = S_0$. Rearranging this one obtains: $\text{E}[S(t)] = S_0e^{rt}$, highlighting why the measure is often described as *risk-neutral*. Under this distribution, investors do not need to be compensated with higher returns for taking on risk and so asset prices are expected to grow at the risk-free rate.

In practise one samples paths of the underlying assuming this unique risk-neutral probability measure. That is, the drift rate of the stochastic process modelling the asset is equal to the risk-free rate: $\mu = r$. The volatility is unchanged in moving to this risk neutral measure. One can then simulate many of these paths and calculate the mean payoff from the derivative at maturity. Finally this average must then be discounted to the present day.

As with Black-Scholes, we will assume that the asset price follows Geometric Brownian Motion. Under the risk-neutral measure this says that $S$ follows the SDE:

$dS = rSdt + \sigma S dW$ where $W$ is a Weiner process. 

A Weiner process is one that, over a small period of time $dt$, changes with distribution: $dW \sim N(0,dt)$. These changes are also independent of one another.

Ito's Lemma can then be used to show that:

$d \ln(S) = (r-\frac{1}{2}\sigma^2)dt +\sigma dW$

Discretising this shows that:

$\ln(S(t+\Delta t)) - \ln(S(t)) = (r-\frac{1}{2}\sigma^2)\Delta t +\sigma \Delta W$

Finally, summing over these differences enables us to directly simulate the price at maturity ($t=T$) from the initial price ($S_0$).

$S(T) = S_0 \exp[(r-\frac{1}{2}\sigma^2)T + \sigma W(T)]$ where $W(T) \sim N(0, T) $

#### Implementation

A confidence interval that contains the true value of the option price 95% of the time has width $\frac{2\times1.96\omega}{\sqrt{n}}$, where $\omega$ is the standard deviation of the simulated prices. Thus, in order for the estimate to be correct to within around \$0.01, $n \approx 10^7$.

#### Assumptions

In using the Fundamental Theorems of Asset Pricing we have implicitly assumed an efficient market in which no arbitrage opportunities exist. Here we have also assumed GBM.

The tree and finite difference methods considered thus far required us to work backwards from the maturity date to the present. Instead Monte Carlo methods simulate paths of the asset price forwards in time. This means that they can be used to price exotic options whereby the payoff depends on the path followed by the underlying (e.g. Asian Options).

Monte Carlo has probably been the slowest pricing method in this simple vanilla case. However, the method is embarassingly parallel and could easily be sped up using multithreading. In addition, Monte Carlo becomes particularly efficient if the payoff depends on multiple underlyings (e.g. a Basket Option) or if the underlying is itself a function of several other, probably correlated, stochastic processes (e.g. fx rates, inflation). In these cases the complexity increases linearly with the number of variables, rather than exponentially for the other two methods.

One drawback of Monte Carlo is that, in simulating forwards, it cannot easily cope with American Options. However, it can most easily incorporate more sophisticated asset pricing models, other than GBM.


In [1]:
import math
import numpy as np

def MCCall(S0, K, r, sigma, T, n):
    
    # n samples from Normal distribution with mean = 0 and variance = T
    Weiner = np.random.normal(0, math.sqrt(T), n)
    
    maturity_values = S0*np.exp( (r - 0.5*math.pow(sigma,2))*T + sigma*Weiner )
    
    maturity_payoff = np.maximum( maturity_values - K, [0 for i in range(n)])
    
    discounted_payoff = np.exp(-r*T)*maturity_payoff
    
    average = sum(discounted_payoff)/n
    
    standard_deviation = np.std(discounted_payoff)
    
    width = 2*1.96*standard_deviation/np.sqrt(n)
    
    print("Confidence Interval Width:", width)
    
    return average

In [2]:
MCCall(200, 205, 0.02, 0.14, 0.5, 10000000)

Confidence Interval Width: 0.0139567604693


6.5589740122926745